In [15]:
from __future__ import absolute_import, division, print_function, unicode_literals

from  IPython import display
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d
import pandas as pd
import numpy as np
import datetime

import tensorflow as tf

!rm -rf ./logs/ 

# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [2]:
higgs_path = tf.keras.utils.get_file('HIGGSSmall.csv.gz', 'https://github.com/PacktWorkshops/The-Reinforcement-Learning-Workshop/blob/master/Chapter03/Dataset/HIGGSSmall.csv.gz?raw=true')

In [3]:
N_TEST = int(1e3)
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(N_TRAIN)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

N_FEATURES = 28

ds = tf.data.experimental.CsvDataset(higgs_path,[float(),]*(N_FEATURES+1), compression_type="GZIP")

def pack_row(*row):
    label = row[0]
    features = tf.stack(row[1:],1)
    return features, label

packed_ds = ds.batch(N_TRAIN).map(pack_row).unbatch()

In [4]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
test_ds = packed_ds.skip(N_VALIDATION).take(N_TEST).cache()
train_ds = packed_ds.skip(N_VALIDATION+N_TEST).take(N_TRAIN).cache()

test_ds = test_ds.batch(BATCH_SIZE)
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

In [5]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

In [6]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

def compile_and_fit(model, name, max_epochs=3000):
    
    optimizer = tf.keras.optimizers.Adam(lr_schedule)
    
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[
                    tf.keras.losses.BinaryCrossentropy(
                        from_logits=True, name='binary_crossentropy'),
                    'accuracy'])

    model.summary()

    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    
    history = model.fit(train_ds,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        epochs=max_epochs,
                        validation_data=validate_ds,
                        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
                                   tensorboard_callback],
                        verbose=2)
    return history

In [ ]:
regularization_model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu', input_shape=(N_FEATURES,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                 activation='elu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

compile_and_fit(regularization_model, "regularizers/regularization", max_epochs=500)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               14848     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               2

Epoch 42/500
20/20 - 1s - loss: 0.7036 - binary_crossentropy: 0.6223 - accuracy: 0.6203 - val_loss: 0.6781 - val_binary_crossentropy: 0.5976 - val_accuracy: 0.6240
Epoch 43/500
20/20 - 1s - loss: 0.6996 - binary_crossentropy: 0.6199 - accuracy: 0.6205 - val_loss: 0.6770 - val_binary_crossentropy: 0.5981 - val_accuracy: 0.6270
Epoch 44/500
20/20 - 1s - loss: 0.7009 - binary_crossentropy: 0.6227 - accuracy: 0.6294 - val_loss: 0.6808 - val_binary_crossentropy: 0.6035 - val_accuracy: 0.6220
Epoch 45/500
20/20 - 1s - loss: 0.7007 - binary_crossentropy: 0.6242 - accuracy: 0.6190 - val_loss: 0.6823 - val_binary_crossentropy: 0.6066 - val_accuracy: 0.6530
Epoch 46/500
20/20 - 1s - loss: 0.6911 - binary_crossentropy: 0.6161 - accuracy: 0.6332 - val_loss: 0.6690 - val_binary_crossentropy: 0.5947 - val_accuracy: 0.6560
Epoch 47/500
20/20 - 1s - loss: 0.6925 - binary_crossentropy: 0.6190 - accuracy: 0.6216 - val_loss: 0.6797 - val_binary_crossentropy: 0.6071 - val_accuracy: 0.6610
Epoch 48/500
20/

Epoch 92/500
20/20 - 1s - loss: 0.6363 - binary_crossentropy: 0.6045 - accuracy: 0.6486 - val_loss: 0.6135 - val_binary_crossentropy: 0.5819 - val_accuracy: 0.6750
Epoch 93/500
20/20 - 1s - loss: 0.6389 - binary_crossentropy: 0.6075 - accuracy: 0.6546 - val_loss: 0.6180 - val_binary_crossentropy: 0.5868 - val_accuracy: 0.6420
Epoch 94/500
20/20 - 1s - loss: 0.6411 - binary_crossentropy: 0.6101 - accuracy: 0.6343 - val_loss: 0.6146 - val_binary_crossentropy: 0.5837 - val_accuracy: 0.6470
Epoch 95/500
20/20 - 1s - loss: 0.6378 - binary_crossentropy: 0.6071 - accuracy: 0.6552 - val_loss: 0.6272 - val_binary_crossentropy: 0.5968 - val_accuracy: 0.6130
Epoch 96/500
20/20 - 1s - loss: 0.6368 - binary_crossentropy: 0.6066 - accuracy: 0.6381 - val_loss: 0.6117 - val_binary_crossentropy: 0.5816 - val_accuracy: 0.6810
Epoch 97/500
20/20 - 1s - loss: 0.6327 - binary_crossentropy: 0.6028 - accuracy: 0.6557 - val_loss: 0.6047 - val_binary_crossentropy: 0.5749 - val_accuracy: 0.6700
Epoch 98/500
20/

Epoch 142/500
20/20 - 1s - loss: 0.6175 - binary_crossentropy: 0.5930 - accuracy: 0.6646 - val_loss: 0.5999 - val_binary_crossentropy: 0.5755 - val_accuracy: 0.6640
Epoch 143/500
20/20 - 1s - loss: 0.6212 - binary_crossentropy: 0.5968 - accuracy: 0.6564 - val_loss: 0.5954 - val_binary_crossentropy: 0.5711 - val_accuracy: 0.6880
Epoch 144/500
20/20 - 1s - loss: 0.6177 - binary_crossentropy: 0.5933 - accuracy: 0.6648 - val_loss: 0.5959 - val_binary_crossentropy: 0.5715 - val_accuracy: 0.6910
Epoch 145/500
20/20 - 1s - loss: 0.6172 - binary_crossentropy: 0.5928 - accuracy: 0.6655 - val_loss: 0.6017 - val_binary_crossentropy: 0.5773 - val_accuracy: 0.6650
Epoch 146/500
20/20 - 1s - loss: 0.6184 - binary_crossentropy: 0.5941 - accuracy: 0.6583 - val_loss: 0.5952 - val_binary_crossentropy: 0.5708 - val_accuracy: 0.6890
Epoch 147/500
20/20 - 1s - loss: 0.6158 - binary_crossentropy: 0.5914 - accuracy: 0.6618 - val_loss: 0.6056 - val_binary_crossentropy: 0.5812 - val_accuracy: 0.6570
Epoch 148/

Epoch 192/500
20/20 - 1s - loss: 0.6144 - binary_crossentropy: 0.5882 - accuracy: 0.6683 - val_loss: 0.5963 - val_binary_crossentropy: 0.5702 - val_accuracy: 0.7010
Epoch 193/500
20/20 - 1s - loss: 0.6128 - binary_crossentropy: 0.5867 - accuracy: 0.6691 - val_loss: 0.5982 - val_binary_crossentropy: 0.5721 - val_accuracy: 0.6780
Epoch 194/500
20/20 - 1s - loss: 0.6175 - binary_crossentropy: 0.5913 - accuracy: 0.6668 - val_loss: 0.6064 - val_binary_crossentropy: 0.5803 - val_accuracy: 0.7020
Epoch 195/500
20/20 - 1s - loss: 0.6163 - binary_crossentropy: 0.5902 - accuracy: 0.6695 - val_loss: 0.6071 - val_binary_crossentropy: 0.5809 - val_accuracy: 0.6290
Epoch 196/500
20/20 - 1s - loss: 0.6143 - binary_crossentropy: 0.5881 - accuracy: 0.6657 - val_loss: 0.5973 - val_binary_crossentropy: 0.5711 - val_accuracy: 0.7030
Epoch 197/500
20/20 - 1s - loss: 0.6151 - binary_crossentropy: 0.5888 - accuracy: 0.6607 - val_loss: 0.6018 - val_binary_crossentropy: 0.5755 - val_accuracy: 0.6620
Epoch 198/

Epoch 242/500
20/20 - 1s - loss: 0.6131 - binary_crossentropy: 0.5854 - accuracy: 0.6726 - val_loss: 0.5921 - val_binary_crossentropy: 0.5644 - val_accuracy: 0.6800
Epoch 243/500
20/20 - 1s - loss: 0.6100 - binary_crossentropy: 0.5823 - accuracy: 0.6753 - val_loss: 0.5940 - val_binary_crossentropy: 0.5662 - val_accuracy: 0.6730
Epoch 244/500
20/20 - 1s - loss: 0.6103 - binary_crossentropy: 0.5825 - accuracy: 0.6743 - val_loss: 0.5915 - val_binary_crossentropy: 0.5638 - val_accuracy: 0.6990
Epoch 245/500
20/20 - 1s - loss: 0.6098 - binary_crossentropy: 0.5821 - accuracy: 0.6706 - val_loss: 0.5976 - val_binary_crossentropy: 0.5698 - val_accuracy: 0.6600
Epoch 246/500
20/20 - 1s - loss: 0.6103 - binary_crossentropy: 0.5825 - accuracy: 0.6728 - val_loss: 0.5932 - val_binary_crossentropy: 0.5653 - val_accuracy: 0.6940
Epoch 247/500
20/20 - 1s - loss: 0.6089 - binary_crossentropy: 0.5811 - accuracy: 0.6739 - val_loss: 0.5916 - val_binary_crossentropy: 0.5638 - val_accuracy: 0.6930
Epoch 248/

Epoch 292/500
20/20 - 1s - loss: 0.6086 - binary_crossentropy: 0.5805 - accuracy: 0.6710 - val_loss: 0.5876 - val_binary_crossentropy: 0.5595 - val_accuracy: 0.6990
Epoch 293/500
20/20 - 1s - loss: 0.6076 - binary_crossentropy: 0.5794 - accuracy: 0.6788 - val_loss: 0.5929 - val_binary_crossentropy: 0.5647 - val_accuracy: 0.6820
Epoch 294/500
20/20 - 1s - loss: 0.6069 - binary_crossentropy: 0.5787 - accuracy: 0.6760 - val_loss: 0.5889 - val_binary_crossentropy: 0.5608 - val_accuracy: 0.6890
Epoch 295/500
20/20 - 1s - loss: 0.6078 - binary_crossentropy: 0.5797 - accuracy: 0.6704 - val_loss: 0.5945 - val_binary_crossentropy: 0.5664 - val_accuracy: 0.6820
Epoch 296/500
20/20 - 1s - loss: 0.6033 - binary_crossentropy: 0.5751 - accuracy: 0.6782 - val_loss: 0.5939 - val_binary_crossentropy: 0.5656 - val_accuracy: 0.6960
Epoch 297/500
20/20 - 1s - loss: 0.6060 - binary_crossentropy: 0.5777 - accuracy: 0.6754 - val_loss: 0.5871 - val_binary_crossentropy: 0.5588 - val_accuracy: 0.6940
Epoch 298/

Epoch 342/500
20/20 - 1s - loss: 0.6077 - binary_crossentropy: 0.5785 - accuracy: 0.6786 - val_loss: 0.5853 - val_binary_crossentropy: 0.5560 - val_accuracy: 0.6910
Epoch 343/500
20/20 - 1s - loss: 0.6049 - binary_crossentropy: 0.5756 - accuracy: 0.6781 - val_loss: 0.5861 - val_binary_crossentropy: 0.5569 - val_accuracy: 0.6910
Epoch 344/500
20/20 - 1s - loss: 0.6033 - binary_crossentropy: 0.5741 - accuracy: 0.6791 - val_loss: 0.5948 - val_binary_crossentropy: 0.5656 - val_accuracy: 0.6610
Epoch 345/500
20/20 - 1s - loss: 0.6059 - binary_crossentropy: 0.5767 - accuracy: 0.6761 - val_loss: 0.5858 - val_binary_crossentropy: 0.5565 - val_accuracy: 0.7050
Epoch 346/500
20/20 - 1s - loss: 0.6031 - binary_crossentropy: 0.5738 - accuracy: 0.6830 - val_loss: 0.5851 - val_binary_crossentropy: 0.5558 - val_accuracy: 0.6930
Epoch 347/500
20/20 - 1s - loss: 0.6046 - binary_crossentropy: 0.5751 - accuracy: 0.6721 - val_loss: 0.5820 - val_binary_crossentropy: 0.5525 - val_accuracy: 0.6980
Epoch 348/

Epoch 392/500


In [13]:
test_accuracy = tf.keras.metrics.Accuracy()

for (features, labels) in test_ds:
    logits = regularization_model(features)
    probabilities = tf.keras.activations.sigmoid(logits)
    predictions = 1*(probabilities.numpy() > 0.5)
    test_accuracy(predictions, labels)
    regularization_model_accuracy = test_accuracy.result()

print("Test set accuracy: {:.3%}".format(regularization_model_accuracy))

Test set accuracy: 68.900%


In [14]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 21724), started 0:00:35 ago. (Use '!kill 21724' to kill it.)